import packages

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from osgeo import gdal_array
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from torch import Tensor, einsum
from torch import nn
from torch.nn import functional as F
from scipy.ndimage import distance_transform_edt as distance
from scipy.spatial.distance import directed_hausdorff
from typing import Any, Callable, Iterable, List, Set, Tuple, TypeVar, Union
 

In [ ]:
!pip install segmentation-models-pytorch==0.1.0
!pip install keras-unet

 

In [ ]:
import segmentation_models_pytorch as smp
from keras_unet.utils import plot_imgs


In [ ]:
size = 256
bs = 16 #batchsize
classes = [ 1, 2, 4,  5,  6,  7,  8,  9, 10, 15]
num_class= len(classes)
EPOCH= 200
reduce = 4

Used to crop the imput images

In [ ]:
def slice (arr, size, inputsize,stride):
    result = []
    if stride is None:
        stride = size
    for i in range(0, (inputsize-size)+1, stride):
        for j in range(0, (inputsize-size)+1, stride):
        
            if arr.ndim == 3:
                s = arr[:,i:(i+size),j:(j+size), ]
            else:
                s = arr[i:(i+size),j:(j+size), ]
            result.append(s)
            #print(i,"",j)
    result = np.array(result)
    return result

def batchslice (arr, size, inputsize, stride, num_img):
    result = []
    for i in range(0, num_img):
        s= slice(arr[i,], size, inputsize, stride )
        result.append(s )
    result = np.array(result)
    result = result.reshape(result.shape[0]*result.shape[1], result.shape[2], result.shape[3], -1)
    return result

def class2dim (mask, CLASSES):
    
        masks = [(mask == v) for v in CLASSES ]
        mask = np.stack(masks, axis=-1).astype('float')    
        return mask




load the different tiles into 1 variable

In [ ]:

#stack all files into 1 variable
def load_raster(path,data,tile1,tile2,reduce):
    tiles_in=[tile1,tile2]
    files = []
    for tile in tiles_in:
       for file in glob.glob(path+data+'*{}.tif'.format(tile)):
           file1 = gdal_array.LoadFile(file)
           #only use 50% of the points to reduce memory
           if np.ndim(file1)==3:
               file1=file1[:,::reduce,::reduce]
           else:
               file1=file1[::reduce,::reduce]
           files.append(file1)
    stacked = np.array(files)
    return stacked

def load_data(path,data1,data2,tile1,tile2, reduce):
    part1 = load_raster(path,data1,tile1,tile2, reduce )
    part2 = load_raster(path,data2,tile1,tile2,reduce )
    
    if np.ndim(part1)< np.ndim(part2):#check if dimmensions are equal
       part1 = np.expand_dims(part1,axis=1)
    elif np.ndim(part1)> np.ndim(part2):
       part2 = np.expand_dims(part2,axis=1)
    print(part1.shape,part2.shape)
    total = np.concatenate((part1,part2),axis=1)
    return total



Slice input data into smaller size images

In [ ]:
training= load_data("../input/guided-research/Training/","DEM","Wadden","12_7","13_8",reduce)
validation = load_data("../input/guided-research/Validation/","DEM","Wadden","12_8","14_7",reduce)
y_train= load_raster("../input/guided-research/Training/","class","12_7","13_8", reduce )
y_val=load_raster("../input/guided-research/Validation/","class","12_8","14_7",reduce )
print(training.shape, validation.shape, y_train.shape, y_val.shape)

In [ ]:
def uniq(a: Tensor) -> Set:
    return set(torch.unique(a.cpu()).numpy())


def sset(a: Tensor, sub: Iterable) -> bool:
    return uniq(a).issubset(sub)

def simplex(t: Tensor, axis=1) -> bool:
    _sum = t.sum(axis).type(torch.float32)
    _ones = torch.ones_like(_sum, dtype=torch.float32)
    return torch.allclose(_sum, _ones)


def one_hot(t: Tensor, axis=1) -> bool:
    return simplex(t, axis) and sset(t, [0, 1])

def one_hot2dist(seg: np.ndarray) -> np.ndarray:
    assert one_hot(torch.Tensor(seg), axis=0)
    C: int = len(seg)

    res = np.zeros_like(seg)
    for c in range(C):
        posmask = seg[c].astype(np.bool)

        if posmask.any():
            negmask = ~posmask
            res[c] = distance(negmask) * negmask - (distance(posmask) - 1) * posmask
    return res

In [ ]:

    

''' 
class GeneralizedDice():
    def __init__(self, **kwargs):
        # Self.idc is used to filter out some classes of the target mask. Use fancy indexing
        self.idc: List[int] = kwargs["idc"]
        print(f"Initialized {self.__class__.__name__} with {kwargs}")

    def __call__(self, probs: Tensor, target: Tensor, _: Tensor) -> Tensor:
        assert simplex(probs) and simplex(target)

        pc = probs[:, self.idc, ...].type(torch.float32)
        tc = target[:, self.idc, ...].type(torch.float32)

        w: Tensor = 1 / ((einsum("bcwh->bc", tc).type(torch.float32) + 1e-10) ** 2)
        intersection: Tensor = w * einsum("bcwh,bcwh->bc", pc, tc)
        union: Tensor = w * (einsum("bcwh->bc", pc) + einsum("bcwh->bc", tc))

        divided: Tensor = 1 - 2 * (einsum("bc->b", intersection) + 1e-10) / (einsum("bc->b", union) + 1e-10)

        loss = divided.mean()

        return loss

class DiceLoss():
    def __init__(self, **kwargs):
        # Self.idc is used to filter out some classes of the target mask. Use fancy indexing
        self.idc: List[int] = kwargs["idc"]
        print(f"Initialized {self.__class__.__name__} with {kwargs}")

    def __call__(self, probs: Tensor, target: Tensor, _: Tensor) -> Tensor:
        assert simplex(probs) and simplex(target)

        pc = probs[:, self.idc, ...].type(torch.float32)
        tc = target[:, self.idc, ...].type(torch.float32)

        intersection: Tensor = einsum("bcwh,bcwh->bc", pc, tc)
        union: Tensor = (einsum("bcwh->bc", pc) + einsum("bcwh->bc", tc))

        divided: Tensor = 1 - (2 * intersection + 1e-10) / (union + 1e-10)

        loss = divided.mean()

        return loss


class SurfaceLoss():
    def __init__(self, **kwargs):
        # Self.idc is used to filter out some classes of the target mask. Use fancy indexing
        self.idc: List[int] = kwargs["idc"]
        print(f"Initialized {self.__class__.__name__} with {kwargs}")

    def __call__(self, probs: Tensor, dist_maps: Tensor, _: Tensor) -> Tensor:
        assert simplex(probs)
        assert not one_hot(dist_maps)

        pc = probs[:, self.idc, ...].type(torch.float32)
        dc = dist_maps[:, self.idc, ...].type(torch.float32)

        multipled = einsum("bcwh,bcwh->bcwh", pc, dc)

        loss = multipled.mean()

        return loss

'''     

In [ ]:
print(training.shape, validation.shape,y_train.shape, y_val.shape)

In [ ]:
'''
plot_imgs(
    org_imgs=training[1:2,0,:,:], # required - original images
    mask_imgs=y_train[1:2,:,:], # required - ground truth masks
    
    nm_img_to_plot=3) # optional - number of images to plot

plot_imgs(
    org_imgs=training[1:2,2,:,:], # required - original images
    mask_imgs=y_train[1:2,:,:], # required - ground truth masks
    
    nm_img_to_plot=3) # optional - number of images to plot
'''

In [ ]:
plot_imgs(
    org_imgs=validation[0:1,0, :,:], # required - original images
    mask_imgs=y_val[0:1,:,:], # required - ground truth masks
    
    nm_img_to_plot=3) # optional - number of images to plot
plot_imgs(
    org_imgs=validation[0:1,1, :,:], # required - original images
    mask_imgs=y_val[0:1,:,:], # required - ground truth masks
    
    nm_img_to_plot=3) # optional - number of images to plot

In [ ]:
#np.moveaxis(validation[0:2,[1,2,3], :,:], 1 , -1)
plot_imgs(
    org_imgs=validation[1:2,0, :,:], # required - original images
    mask_imgs=y_val[1:2,:,:], # required - ground truth masks
    
    nm_img_to_plot=3) # optional - number of images to plot
plot_imgs(
    org_imgs=validation[1:2,1, :,:], # required - original images
    mask_imgs=y_val[1:2,:,:], # required - ground truth masks
    
    nm_img_to_plot=3) # optional - number of images to plot

In [ ]:
#a= np.moveaxis(validation[1:2 ,1:4,:,:], 1, -1)
# required - original images
 
multival = class2dim(y_val, classes) 
for i in range(len(classes)):
    plot_imgs(
    org_imgs=validation[0:1, 2, :,: ],
    mask_imgs=multival[0:1,:,:,i], # required - original images 

    nm_img_to_plot=10) # optional - number of images to plot


In [ ]:
training  = batchslice(training,size,training[0].shape[1],size,training.shape[0])
validation=batchslice(validation,size,validation[0].shape[1],size,validation.shape[0])

y_train = batchslice(y_train,size,y_train[0].shape[1],size,y_train.shape[0]).squeeze()
y_val = batchslice(y_val,size,y_val[0].shape[1],size,y_val.shape[0]).squeeze()
  

print(training.shape,validation.shape,y_train.shape, y_val.shape)


In [ ]:
#plt.itraining= load_data("../input/guided-research/Training/","DEM","Wadden","12_7","13_8")mshow(validation[1,0,:,:])
#plt.show() 
plot_imgs(
    org_imgs=validation[0:9,1, :,:], # required - original images
    mask_imgs=y_val[0:9,:,:], # required - ground truth masks
    
    nm_img_to_plot=10) # optional - number of images to plot

In [ ]:
#for wh in range(10,20):
#    plt.imshow(validation[wh,0,:,:])
#    plt.show() 
#    plt.imshow(validation[wh,2,:,:])
#    plt.show() 
#    plt.imshow(y_val[wh,:,:])
#    plt.show()  

In [ ]:
new_mask = class2dim(y_train, classes)
new_mas_val = class2dim(y_val, classes)

In [ ]:
#check
#whichone = 5
#print(np.amax(y_train[whichone,] ),np.amin(y_train[whichone,] ))
#plt.imshow(y_train[whichone,] )
#plt.show() 
#for n in range(num_class): 
#    plt.imshow(new_mask[whichone,:,:,n])     
#    plt.show() 


Code used to see slicing result

In [ ]:
new_mask =  np.moveaxis(new_mask, -1, 1)
new_mas_val = np.moveaxis(new_mas_val, -1, 1)

In [ ]:
new_mask.shape

In [ ]:
plt.imshow(new_mas_val[0,1,])
a = one_hot2dist(new_mas_val[0,])
plt.imshow(a[1,])

In [ ]:
training[training> 1e308] = 0 
#np.nan
validation[validation> 1e308] =0 
new_mask[new_mask >1e308] = 0
new_mas_val[new_mas_val> 1e308] = 0

training = list(zip(training[:,1,:,:], y_train))
validation = list(zip(validation[:,1,:,:], y_val))m
 

In [ ]:
#np.save("../wadden1", zip(training, new_mask, validation, new_mas_val))

Get torch loader, got the error in catalyst with "collection dont have attribute size", use original array as demonstrated in catalyst

In [ ]:
#net = torch.hub.load('milesial/Pytorch-UNet', 'unet_carvana')
#print(net)

In [ ]:
#CLASSES = [format(x, '2d') for x in  classes]
 
#model = smp.Unet(classes=len(CLASSES), in_channels=3,activation="sigmoid")
#newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
#print(model)


In [ ]:
#model = net
 

In [ ]:
#comparison = training == validation
#equal_arrays = comparison.all()
#print(equal_arrays)
#inputs, masks = next(iter(loaders['valid']))
 

Pre-trained Unet segmentation model

In [ ]:
from catalyst.dl import utils
import collections
from catalyst.data import Augmentor
import torchvision.transforms as transforms

open_fn = lambda x: {"features": x[0], "targets": x[1]}
data_transform = transforms.Compose([
    Augmentor(
        dict_key="features",
        augment_fn=lambda x: \
            torch.from_numpy(x.copy().astype(np.float32) / 1.)),
  #  Augmentor(
  #      dict_key="features",
  #      augment_fn=transforms.Normalize(
  #          (0.5, ),
  #          (0.5, ))),
    Augmentor(
        dict_key="targets",
        augment_fn=lambda x: \
            torch.from_numpy(x.copy().astype(np.float32) / 1.) )
])
#.unsqueeze_(0)


train_loader = utils.get_loader(
    list(zip(training.astype(np.float32), new_mask.astype(np.float32))), 
    open_fn=open_fn,  
    batch_size=bs, 
    dict_transform=data_transform, 
    num_workers=4, 
    shuffle=True)

valid_loader = utils.get_loader(
    list(zip(validation.astype(np.float32), new_mas_val.astype(np.float32))), 
    open_fn=open_fn,  
    batch_size=bs, 
    dict_transform=data_transform, 
    num_workers=4, 
    shuffle=False)

 
from catalyst.dl import utils
 
loaders = collections.OrderedDict()


loaders["train"] = train_loader
loaders["valid"] =valid_loader

In [ ]:
'''
for i, inputs in enumerate (loaders['train']):          
         print (inputs['features'].shape)
'''

%load_ext tensorboard
%tensorboard --logdir {logdir}

In [ ]:
GAMMA = 2
ALPHA = 0.8 # emphasize FP
BETA = 0.2 # more emphasize on FN

In [ ]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE


     
class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True, **kwargs):
        super(IoULoss, self).__init__(**kwargs)

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions 
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth)
                
        return 1 - IoU
    
class TverskyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(TverskyLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()    
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()
       
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        
        return 1 - Tversky
 

class FocalTverskyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalTverskyLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA, gamma=GAMMA):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()    
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()
        
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        FocalTversky = (1 - Tversky)**gamma
                       
        return FocalTversky

class myLoss(torch.nn.Module):

    def __init__(self, pos_weight=1):
      super().__init__()
      self.pos_weight = pos_weight

    def forward(self, input, target):
      epsilon = 10 ** -44
      input = input.sigmoid().clamp(epsilon, 1 - epsilon)

      my_bce_loss = -1 * (self.pos_weight * target * torch.log(input)
                          + (1 - target) * torch.log(1 - input))
      add_loss = (target - 0.5) ** 2 * 4
      mean_loss = (my_bce_loss * add_loss).mean()
      return mean_loss
    
 

class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss
    

In [ ]:
from catalyst.contrib.nn import DiceLoss, IoULoss
from catalyst.dl.callbacks import DiceCallback, IouCallback, \
  CriterionCallback, MetricAggregationCallback
# we have multiple criterions
criterion = {
    "dice": DiceLoss(),
    "iou": IoULoss(),
    "bce": nn.BCEWithLogitsLoss()
}


callbacks = [
    # Each criterion is calculated separately.
    CriterionCallback(
        input_key="mask",
        prefix="loss_dice",
        criterion_key="dice"
    ),
    CriterionCallback(
        input_key="mask",
        prefix="loss_iou",
        criterion_key="iou"
    ),
    CriterionCallback(
        input_key="mask",
        prefix="loss_bce",
        criterion_key="bce"
    ),

    # And only then we aggregate everything into one loss.
    MetricAggregationCallback(
        prefix="loss",
        mode="weighted_sum", # can be "sum", "weighted_sum" or "mean"
        # because we want weighted sum, we need to add scale for each loss
        metrics={"loss_dice": 1.0, "loss_iou": 1.0, "loss_bce": 0.8},
    ),

    # metrics
    DiceCallback(input_key="mask"),
    IouCallback(input_key="mask"),
]

 

In [ ]:
#from catalyst.contrib.models.cv import Unet,ResnetUnet
#from catalyst.contrib.nn import DiceLoss, IoULoss, NLLLoss

from catalyst.dl import SupervisedRunner

# experiment setup
num_epochs =EPOCH
logdir = "segmentation_notebook"

# model, criterion, optimizer
#model =  torchvision.models.segmentation.fcn_resnet50(pretrained=False, progress=True, num_classes=num_class)
#model =Unet(num_classes=num_class, in_channels=4, num_channels=64, num_blocks=4)# catalyst Unet
#model = ResnetUnet(num_classes=num_class) # can only take 3 channels 
model = smp.Unet(classes=num_class, in_channels=4, activation = None)
 
criterion = FocalTverskyLoss()
#"sigmoid" nn.BCELoss()
#smp.utils.losses.DiceLoss()
#WithLogits 
#myLoss()
#nn.BCEWithLogitsLoss() # this and myLoss() can run but gradient does not descent. 
# nn.CrossEntropyLoss()  # the type should be Long
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 40], gamma=0.3)

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    #callbacks=callbacks,
    logdir=logdir,
    num_epochs=num_epochs,
    verbose=True,
    main_metric='iou',
    callbacks=[ 
        IouCallback(),
    ],
    load_best_on_end=True,
)
 

In [ ]:
#import inspect
#inspect.signature(runner.train)

In [ ]:
f"{logdir}//checkpoints/best_full.pth"

In [ ]:
 
#torch.load(f"{logdir}//checkpoints/best_full.pth")  # Choose whatever GPU device number you want
 

In [ ]:
sigmoid = lambda x: 1/(1 + np.exp(-x))


In [ ]:
runner_out = runner.predict_loader(loader=loaders["valid"])

for i, output in enumerate (runner_out):   
    if i == 0: 
        
        output =  output["logits"].cpu() 
        output = (output> .5).detach().numpy().astype(int)
        for j in range (bs):
            #print(f"{i} th batch {j} th image")
            
            plt.figure(figsize=(10,10))
            output2 = np.sum(output[j,], axis = 0)
       
            plt.subplot(1 ,5, 1 )
            
            plt.imshow(output2, cmap=plt.get_cmap('Set3'))
            plt.title('prediction')
            plt.legend(f"{j}")
            plt.subplot(1 ,5, 2 )
            plt.imshow(validation[j,2,])
            plt.title('R-band') 
            plt.subplot(1 ,5, 3 )
            plt.imshow(validation[j,1,])
            plt.title('NIR-band') 
            plt.subplot(1 ,5, 4)
            plt.imshow(validation[j,0,])
            plt.title('LIDAR-band') 
            plt.subplot(1 ,5, 5)
            plt.imshow(y_val[j,]) 
            plt.title('validation')
            

In [ ]:
wc = 4
runner_out = runner.predict_loader(loader=loaders["valid"])

for i, output in enumerate (runner_out):   
    if i == 3: 
        
        output =  output["logits"].cpu() 
        output = (output > .8).detach().numpy().astype(int)
        for j in range (bs):
            #print(f"{i} th batch {j} th image")
            
            plt.figure(figsize=(10,10))
            
            output2 = output[j, wc, ]#np.sum(output[j,], axis = 0).astype(bool).astype(int)
       
            plt.subplot(1 ,5, 1 )
            
            plt.imshow(output2)
            plt.title(f'{wc}-th class pred')
            plt.legend(f"{j}")
            plt.subplot(1 ,5, 2 )
            plt.imshow(validation[i*16+j,2,])
            plt.title('R-band') 
            plt.subplot(1 ,5, 3 )
            plt.imshow(validation[i*16+j,1,])
            plt.title('NIR-band') 
            plt.subplot(1 ,5, 4)
            plt.imshow(validation[i*16+j,0,])
            plt.title('LIDAR-band') 
            plt.subplot(1 ,5, 5)
            plt.imshow(y_val[i*16+j,]) 
            plt.title('validation')
            plt.legend()

            plt.show()

In [ ]:
classification_tile128 = gdal_array.LoadFile('../input/guided-research/Validation/class_main_12_8.tif')
classification_tile128=classification_tile128[::reduce,::reduce]
numimag = np.sqrt(validation.shape[0]/2).astype(int)
num = (numimag*numimag+1).astype(int)
#fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,8)) 
#for ii,axi in enumerate(ax.flat):
classes = [1,2,4,5,6,7,8,9,10,15]
for wc in range (10) :
    runner_out = runner.predict_loader(loader=loaders["valid"])
    #for the first tile only
    result = np.zeros((1,size, size))
    for i, output in enumerate (runner_out):
            output1 =  output["logits"].cpu()
            output1 = (output1   > .5).detach().numpy().astype(int)
            p1 = output1[:,wc,] #np.sum(output1[0,], axis = 0).astype(bool).astype(int)    

            result = np.concatenate((result, p1),axis=0)
           # print(result.shape)     
    result = result[1:num,:,:]

    result= np.moveaxis(result, 0, -1)

    big = np.zeros((numimag*size, numimag*size))

    for j in range(numimag):    
        for i in range(numimag):
            big[j*size: (j+1)*size,i*size: (i+1)*size]= result[:,:,i+j*numimag]


    plt.subplot(1,2,1)
    plt.title('prediction')
    plt.imshow(big)
    plt.subplot(1,2,2)
    plt.imshow(classification_tile128==classes[wc])
    plt.title(f'validation class: {classes[wc]}')
    plt.show()

In [ ]:
classification_tile128 = gdal_array.LoadFile('../input/guided-research/Validation/class_main_14_7.tif')
classification_tile128=classification_tile128[::reduce,::reduce] 
for wc in range (10) :
    runner_out = runner.predict_loader(loader=loaders["valid"])
    #for the first tile only
    result = np.zeros((1,size, size))
    for i, output in enumerate (runner_out):
            output1 =  output["logits"].cpu()
            output1 = (output1   > .5).detach().numpy().astype(int)
            p1 = output1[:,wc,] #np.sum(output1[0,], axis = 0).astype(bool).astype(int)    

            result = np.concatenate((result, p1),axis=0)
           # print(result.shape)     
    result = result[num:,:,:]

    result= np.moveaxis(result, 0, -1)

    big = np.zeros((numimag*size, numimag*size))

    for j in range(numimag):    
        for i in range(numimag):
            big[j*size: (j+1)*size,i*size: (i+1)*size]= result[:,:,i+j*numimag]


    plt.subplot(1,2,1)
    plt.title('prediction')
    plt.imshow(big)
    plt.subplot(1,2,2)
    plt.imshow(classification_tile128==classes[wc])
    plt.title(f'validation class: {classes[wc]}')
    plt.show()

In [ ]:
'''
torch.save(model.state_dict(),'model.pt')
for wc in range (10) :
    def extra(i,data):
        if i == 0:
            d = data[15,wc,]#np.sum(data[15,],axis=0).astype(bool).astype(int)
        else:
            d=[]
            for x in range(15-i,16):
                d.append(data[x,wc,])
            d = np.concatenate(d,axis=1)
        return d      

    runner_out = runner.predict_loader(loader=loaders["valid"])
    number=1
    a= np.empty([])
    tot=np.empty((128*2,1920*2))
    xx =0
    #for the first tile only
    for i, output in enumerate (runner_out):
        if i in range(0,15):#batches
            #print(f"{i} th batch")
            output1 =  output["logits"].cpu()
            output1 = (sigmoid(output1)  > .7).detach().numpy().astype(int)
            p1 = output1[0,wc,] #np.sum(output1[0,], axis = 0).astype(bool).astype(int)
            
            number+=1
            if xx>0 and xx!=14:
                p1 = np.append(next_row,p1,axis=1)
                number+=(1*(xx+1))
            if xx == 14:
                p1 = np.append(next_row,p1,axis=1)
                number+=1
            else:
                for j in range (1,15-xx):#images in 1 batch
                    output2 = output1[j,wc,]#np.sum(output1[j,], axis = 0).astype(bool).astype(int)
                    p1=np.append(p1,output2,axis=1)
                    number+=1
                next_row = extra(xx,output1)
                xx+=1
        else:
            continue
        tot = np.append(tot,p1,axis=0)

        #print('tot',tot.shape,number)



    tot=tot[128*2:,:]
    #print(tot.shape[0]/128,tot.shape[1]/128,number)  
    plt.imshow(tot)
    plt.title(f'class: {wc}')
    plt.show()
'''

In [ ]:
''' # probabilistic
runner_out = runner.predict_loader(loader=loaders["valid"])
wh = 2
plt.figure(figsize=(18,18))
for i, output in enumerate (runner_out):   
    if i == 0: 
        print(i)
        for j in range(1,10): 
            
            plt.subplot(10,3, 1+3*(j-1))
            plt.imshow(output['logits'].cpu()[wh,j,])
            plt.subplot(10,3, 2+3*(j-1))
            plt.imshow(validation[wh,2,])
            plt.subplot(10, 3, 3+3*(j-1))
            plt.imshow(new_mas_val[wh,j,]) 
''' 
'''

In [ ]:

''' plot
#plt.style.use("ggplot")
#%matplotlib inline

sigmoid = lambda x: 1/(1 + np.exp(-x))

for i, (input, output) in enumerate(zip(validation, runner_out)):
    image, mask = input
    
    threshold = 0.5
    
    plt.figure(figsize=(10,8))
    
    
    plt.imshow(image, 'gray')
    
    plt.subplot(1, 3, 2)
    output =  output["logits"].copy()
    output = (output > threshold).astype(np.uint8)
    plt.imshow(output, "gray")
    
    plt.subplot(1, 3, 3)
    plt.imshow(mask, "gray")
    
    plt.show()
'''

How to update a model
https://heartbeat.fritz.ai/image-segmentation-with-transfer-learning-pytorch-5ada7121c6ab

In [ ]:
'''#for loop way of class2dimm
def class2dim2 (mask, num_class):
    
    new_mask = np.reshape(np.tile(mask, num_class), ( mask.shape[0], mask.shape[1],mask.shape[2], num_class))
 
    for j in range(mask.shape[0]):
         aslice = new_mask[j,]
         amask = mask[j,]
         for i in range(num_class):
             aslice[:,:,i] = np.multiply(amask == i+1,1)        
         new_mask[j,]=aslice
    return new_mask
    
a = class2dim2(y_train, num_class)
a.shape
comparison = a == new_mask
equal_arrays = comparison.all()
print(equal_arrays)
'''




In [ ]:
''' # Not using catalyst, the lost is a different format 

from torch.utils.data import DataLoader, TensorDataset 
def myloader(trainX, trainY, valX,valY):
    train = TensorDataset(torch.Tensor(trainX[:,0:3,:,:]), torch.Tensor(trainY )) # create your datset
    train  = DataLoader(train, batch_size=bs) # create your dataloader
    
    vali= TensorDataset(torch.Tensor(valX[:,0:3,:,:]),torch.Tensor(valY  )) # create your datset
    vali = DataLoader(vali, batch_size=bs) # create your dataloader
    return train , vali 

train_loader, valid_loader = myloader(training, new_mask, validation, new_mas_val)
 # torch.LongTensor() for some losses
CLASSES = [format(x, '2d') for x in  classes]
 
model = smp.Unet(classes=len(CLASSES), in_channels=3,activation="sigmoid")
#newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
#print(model)

ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'

print(len(CLASSES))
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
#model = smp.FPN(
#    encoder_name=ENCODER, 
#    encoder_weights=ENCODER_WEIGHTS, 
#    classes=len(CLASSES), 
#    activation=ACTIVATION,
#)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

loss = smp.utils.losses.DiceLoss()
#loss=nn.BCELoss(reduction = 'sum')
#WithLogits
#loss = nn.BCELoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)
max_score = 0

for i in range(0, EPOCH):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, 'best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5

        print('Decrease decoder learning rate to 1e-5!')

best_model = torch.load('./best_model.pth')
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
for i in range(10):
    x_tensor = torch.from_numpy(validation[:,0:3,:,:]).to(DEVICE)     
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy())
        
    visualize( 
        predicted_mask=pr_mask[2,i,:,:]
    )
plt.imshow(validation[2,1,:,:])
plt.show()
plt.imshow(y_val[2,:,:])
''' 